In [1]:
import sys
import os
import glob
import torch
import matplotlib.pyplot as plt
import pandas as pd
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
df = pd.read_csv("/app/dataset/metadata/annoated.csv",sep="	")
ROOT_DIR="/app/dataset/examples/"

In [16]:
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from torch.utils.data import DataLoader
from tqdm import tqdm
from configs.whisper_characterwise import Configs
files = df.file.apply(lambda x: os.path.join(ROOT_DIR,x))
CFG = Configs(files,df.sentence)

inference mode is on


In [18]:
data_loader = DataLoader(CFG.inference_dataset,batch_size=64, pin_memory=True, num_workers=4)

In [19]:
device = "cuda"
CFG.load_state_dict("/app/bengali-speech-recognition/workdir/whisperbase_characterlevel/bestmodel_wer.pkl")
CFG.model.to(device)
1

loading model checkpoint from epoch:  14


1

In [20]:
def infer(inputs):
    batch_size = inputs.size(0)
    generated_tokens = torch.ones((batch_size, 1), dtype=torch.long, device=device) * CFG.START_TOKEN
    encoded_logits = CFG.model.encoder(inputs)
    eos_flags = torch.zeros(batch_size, dtype=torch.bool, device=device)

    for _ in range(CFG.MAX_PREDICTION_LENGTH):
        logits = CFG.model.decoder(generated_tokens, encoded_logits)
        next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        generated_tokens = torch.cat([generated_tokens, next_token], dim=1)

        # Update end-of-sequence flags
        eos_flags = eos_flags | (next_token.squeeze(-1) == CFG.END_TOKEN)

        # Stop condition: if all sequences in the batch have generated <eos>
        if eos_flags.all():
            break
    return generated_tokens

In [21]:
from jiwer import wer, cer

In [24]:
total_wer = 0
total_cer = 0
total_samples = 0

truths = []
predictions = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        inputs, _, target_tokens = batch

        # Initialize tokens (assuming <sos> token is 0)
        generated_tokens = infer(inputs.to(device)).detach().cpu()

        generated_tokens = generated_tokens[:, 1:]  # Remove the start token
        for gen,tar in zip(generated_tokens,target_tokens):
            end_pos = (gen == CFG.END_TOKEN).nonzero(as_tuple=True)[0]
            if len(end_pos) > 0:
                gen = gen[:end_pos[0]] 
            hypothesis = CFG.tokenizer.decode_torch_inference(gen)
            reference = CFG.tokenizer.decode_torch_inference(tar[tar!=CFG.PAD_TOKEN])

            predictions.append(hypothesis)
            truths.append(reference)

            # total_wer += wer(reference, hypothesis)
            # total_cer += cer(reference, hypothesis)
            # total_samples += 1
# avg_wer = total_wer / total_samples
# avg_cer = total_cer / total_samples
# print(avg_wer,avg_cer)

100%|██████████| 1/1 [00:08<00:00,  8.27s/it]

0.9894057188531702 0.8006513027137135


In [25]:
predictions

['সিনেমার ক্রিকেট সংগ্রহের কেপ টেবিল স্টেট বিশ্ববিদ্যালয়ের প্রথম ডাক্তার হিসেবে বর্ণনা করবেন বিশ্ববিদ্যালয়ের প্রথম ডাক্তার সাবেক ডেডব্লিং ডিরালবের সবক্তিশায়ীবিদ্যতায় সবিদীবিত সবিনিনিদ্বববিদেবিদ্বববিনাবিদ্ববিনাবংবংববংবংবংবংবংবংবংবংবংবংবংবংবংবং বং বং বং বং',
 'তবে বিশ্ববিদ্যালয়ের প্রথম বিশ্ববিদ্যালয়ের প্রথম বিশ্ববিদ্যালয়টি ফিল্ড শিক্ষা বিভাগের শিক্ষক পদবী বিভাগের শিক্ষক সিলিন আব্দুল ফিল্ড ফিল্বে পাঠাক ছিলেন।',
 'আমি তোমাকে প্রিয়াংকা বলে দিবো আমি তোমাকে বিশ্বাস করবো না তোমাকে ভালোবাসি।',
 'তবে প্রথম বিশ্বকাপ শুরু হবে বাংলাদেশ প্রথম পার্বলিউড স্পেনের প্রথম প্রথম মুসলিম।',
 'তিনি সার্কাস ও সিলেট সিলেট সিলেট সিলেট সার্কাসে স্বেচ্ছাবিদ সিলেট স্কটল্যান্ড বিশ্ববিদ্যালয়ের সাংবাদিক বিচার সম্পর্কে বিচার করেন, যদিল সিলাত সিলাত স্বাদ বিশিরববদে সিয়েন।',
 'তবে বাংলাদেশ ব্যাংক ব্যাংক ব্যাংক ব্যাংক ব্যাংক ব্যাংক ব্যাংক সেবা পাওয়ার সিদ্ধান্ত নেওয়া হয়েছে।',
 'এখানকার প্রধান পরিবেশ দ্বারপ্রাপ্ত প্রধান শিল্প প্রাক্তন স্থান প্রদর্শন করে এবং বাংলাদেশে শিল্প ব্যবস্থা সংক্রান্ত বিষয়ক বিভাগে স্নাতক ডিগ